In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

"""load and prepare data"""
#read raw data into pandas Dataframe 
df_base = pd.read_excel("C:\\Users\\anna-\\Nextcloud2\\DHBW\\2. Sem\\DataScienceProject\\dataexport_20220730T081818.xlsx", engine='openpyxl')
#drop first 9 unneeded columns
df_base = df_base.drop(labels = np.arange(0,9), axis = 0)
#reset index (starting from 0)
df_base = df_base.reset_index(drop=True)
#rename columns
df_base.set_axis(["day","precipitation","temp_min","temp_max"], axis=1, inplace=True)
#format timestamp in raw data and insert new columns for month and day 
df_base.insert(0, "month", df_base["day"].astype('string').str.replace('00:00:00','').str.split('-').str[1].astype('int'))
df_base.insert(0, "year", df_base["day"].astype('string').str.replace('00:00:00','').str.split('-').str[0].astype('int'))
df_base["day"] = df_base["day"].astype('string').str.replace('00:00:00','').str.split('-').str[2].astype('int')
df_base = df_base.dropna(axis=0)


C:\Users\anna-\anaconda3\envs\TSA\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [2]:
df_base

,year,month,day,precipitation,temp_min,temp_max
0,1984,1,1,0,-1.63,4.73
1,1984,1,2,0,-1.83,3.4
2,1984,1,3,8.400001,0.71,7.77
3,1984,1,4,4.0,1.38,2.96
4,1984,1,5,0.9,-1.75,2.12
...,...,...,...,...,...,...
14078,2022,7,18,0,14.02,34.8
14079,2022,7,19,0,16.05,36.38
14080,2022,7,20,8.8,19.27,32.96
14081,2022,7,21,0,17.48,28.73


In [3]:
df_base["date"] = pd.to_datetime(df_base.year.astype(str)+'-'+df_base.month.astype(str)+'-'+df_base.day.astype(str), format="%Y-%m-%d")

In [4]:
from darts import TimeSeries
from darts.models.forecasting.auto_arima import AutoARIMA
from darts.models.forecasting.arima import ARIMA
from darts.models.forecasting.rnn_model import RNNModel

n = 365

ts = TimeSeries.from_dataframe(df_base, time_col="date", value_cols="temp_max")



C:\Users\anna-\anaconda3\envs\TSA\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
model = RNNModel(128, n_epochs=100)
model.fit(ts)

2022-08-09 08:41:34 darts.models.forecasting.torch_forecasting_model INFO: Train dataset contains 14059 samples.
2022-08-09 08:41:34 darts.models.forecasting.torch_forecasting_model INFO: Time series values are 64-bits; casting model to float64.
2022-08-09 08:41:34 pytorch_lightning.utilities.rank_zero INFO: GPU available: False, used: False
2022-08-09 08:41:34 pytorch_lightning.utilities.rank_zero INFO: TPU available: False, using: 0 TPU cores
2022-08-09 08:41:34 pytorch_lightning.utilities.rank_zero INFO: IPU available: False, using: 0 IPUs
2022-08-09 08:41:34 pytorch_lightning.utilities.rank_zero INFO: HPU available: False, using: 0 HPUs
2022-08-09 08:41:34 pytorch_lightning.callbacks.model_summary INFO: 
  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | RNN              | 700   


Epoch 99: 100%|█████████████████████████████████████████| 440/440 [00:16<00:00, 27.19it/s, loss=8.35, train_loss=10.00]

2022-08-09 09:08:56 pytorch_lightning.utilities.rank_zero INFO: `Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|█████████████████████████████████████████| 440/440 [00:16<00:00, 27.18it/s, loss=8.35, train_loss=10.00]


In [23]:
y_pred=model.predict(n)

Predicting DataLoader 0: 100%|███████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.57it/s]


In [24]:
import plotly.express as px

In [25]:
import numpy as np

In [26]:
x = np.concatenate([df_base.date.to_numpy(),y_pred.time_index.values])

In [27]:
df_base.shape

(14083, 7)

In [28]:
y = np.concatenate([df_base.temp_max.to_numpy(),y_pred.values().reshape(-1)])

In [29]:
plotdf = pd.DataFrame({"x":df_base.date.to_numpy(),"y":df_base.temp_max.to_numpy()})
plotdf["type"] = "original"

In [30]:
plotdf_pred = pd.DataFrame({"x":y_pred.time_index.values,"y":y_pred.values().reshape(-1)})
plotdf_pred["type"] = "prediction"

In [31]:
plotdf=pd.concat([plotdf,plotdf_pred])

In [32]:
plotdf

,x,y,type
0,1984-01-01,4.73,original
1,1984-01-02,3.4,original
2,1984-01-03,7.77,original
3,1984-01-04,2.96,original
4,1984-01-05,2.12,original
...,...,...,...
360,2023-07-18,5.256106,prediction
361,2023-07-19,5.256106,prediction
362,2023-07-20,5.256106,prediction
363,2023-07-21,5.256106,prediction


In [33]:
y.shape,x.shape

((14448,), (14448,))

In [34]:
px.scatter(plotdf,x="x",y="y",color="type")

In [35]:
y_pred


<TimeSeries (DataArray) (date: 365, component: 1, sample: 1)>
array([[[31.20669691]],

       [[28.41903566]],

       [[27.67036047]],

       [[26.93343627]],

       [[26.08337093]],

       [[25.17888638]],

       [[24.28065128]],

       [[23.37507103]],

       [[22.54908066]],

       [[21.97854051]],

...

       [[ 5.25610553]],

       [[ 5.25610553]],

       [[ 5.25610553]],

       [[ 5.25610553]],

       [[ 5.25610553]],

       [[ 5.25610553]],

       [[ 5.25610553]],

       [[ 5.25610553]],

       [[ 5.25610553]],

       [[ 5.25610553]]])
Coordinates:
  * date       (date) datetime64[ns] 2022-07-23 2022-07-24 ... 2023-07-22
  * component  (component) object 'temp_max'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [36]:
y_pred.values().reshape(-1)

array([31.20669691, 28.41903566, 27.67036047, 26.93343627, 26.08337093,
       25.17888638, 24.28065128, 23.37507103, 22.54908066, 21.97854051,
       21.62410207, 21.35979033, 21.12109596, 20.87290134, 20.60339355,
       20.31631114, 20.0154833 , 19.70409705, 19.38817031, 19.0767109 ,
       18.77883063, 18.50129551, 18.24745603, 18.0172574 , 17.80792552,
       17.61496151, 17.43303842, 17.25654691, 17.07976937, 16.89679678,
       16.70133474, 16.48657467, 16.24541634, 15.97153702, 15.66192099,
       15.32075152, 14.96204985, 14.60609893, 14.26921244, 13.95541484,
       13.6561221 , 13.35391265, 13.0252466 , 12.6418281 , 12.17720561,
       11.63299958, 11.07502465, 10.59352266, 10.2166315 ,  9.92979256,
        9.70769534,  9.52254866,  9.35273924,  9.18659717,  9.02022046,
        8.85345325,  8.68763258,  8.52447901,  8.36542399,  8.21136063,
        8.06271218,  7.91957058,  7.78181262,  7.64918787,  7.52138205,
        7.39805978,  7.27889492,  7.16359245,  7.05190363,  6.94

In [37]:
y_pred

<TimeSeries (DataArray) (date: 365, component: 1, sample: 1)>
array([[[31.20669691]],

       [[28.41903566]],

       [[27.67036047]],

       [[26.93343627]],

       [[26.08337093]],

       [[25.17888638]],

       [[24.28065128]],

       [[23.37507103]],

       [[22.54908066]],

       [[21.97854051]],

...

       [[ 5.25610553]],

       [[ 5.25610553]],

       [[ 5.25610553]],

       [[ 5.25610553]],

       [[ 5.25610553]],

       [[ 5.25610553]],

       [[ 5.25610553]],

       [[ 5.25610553]],

       [[ 5.25610553]],

       [[ 5.25610553]]])
Coordinates:
  * date       (date) datetime64[ns] 2022-07-23 2022-07-24 ... 2023-07-22
  * component  (component) object 'temp_max'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None